In [1]:
import os
from pathlib import Path
import re
from datetime import datetime
from configparser import ConfigParser
import psycopg2
from psycopg2.extensions import AsIs
import pandas as pd


In [3]:
filename = Path(os.path.expanduser('~')) / ".database.ini"
section = 'iucnecoadm'

parser = ConfigParser()
parser.read(filename)
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

params = db

In [4]:
inputdir = Path(os.path.expanduser('~')) / "tmp" / "typology-web-update-content"
filename = 'efg-short-descriptions.ods'
df = pd.read_excel(inputdir / filename, engine='odf')

In [5]:
df

,code,name,keyfeatures,distdesc
0,T1.1,T1.1 Tropical-subtropical lowland rainforests,Tall closed-canopy evergreen forests in warm w...,Global wet tropics & subtropics
1,T1.2,T1.2 Tropical-subtropical dry forests and thic...,Closed-canopy deciduous and semi-deciduous for...,Global wet/dry tropics & subtropics
2,T1.3,T1.3 Tropical-subtropical montane rainforests,Closed-canopy evergreen forests with abundant ...,Global tropical & subtropical mountains
3,T1.4,T1.4 Tropical heath forests,Low closed-canopy evergreen forests in warm we...,"Amazon basin, southeast Asia, possibly Congo b..."
4,T2.1,T2.1 Boreal and temperate montane forests and ...,"Closed to open, evergreen (conifers) or decidu...",Cool regions (boreal zone or mountains in temp...
...,...,...,...,...
105,MT2.2,MT2.2 Large seabird and pinniped colonies,Localised areas of bare or vegetated ground wi...,"Scattered globally on islands and coastlines, ..."
106,MT3.1,MT3.1 Artificial shores,"Coastal infrastructure, such as seawalls, brea...","Globally, along urbanised coastlines"
107,MFT1.1,MFT 1.1 Coastal river deltas,"Depositional, mosaic systems with strong gradi...",Continental margins of high rainfall catchment...
108,MFT1.2,MFT1.2 Intertidal forests and shrublands,"Intertidal mangrove-dominated systems, produci...",Tropical and warm temperate coastlines with go...


In [30]:
conn = psycopg2.connect(**db)
cur = conn.cursor()

In [31]:
updstr="""UPDATE functional_groups set keyfeatures=%s,distdesc=%s where code=%s;"""
for row in df.values:
    cur.execute(updstr,(row[2].strip(),row[3].strip(),row[0]))

In [32]:
cur.close()
conn.commit()
if conn is not None:
    conn.close()
    print('Database connection closed.')

Database connection closed.


In [28]:
updstr

'UPDATE functional_groups set keyfeatures=%s,distdesc=%s where code=%s'

In [8]:
df.values

array([['T1.1', 'T1.1 Tropical-subtropical lowland rainforests\xa0',
        'Tall closed-canopy evergreen forests in warm wet climates, phylogenetically & functionally highly diverse life forms\xa0',
        'Global wet tropics & subtropics\xa0'],
       ['T1.2', 'T1.2 Tropical-subtropical dry forests and thickets\xa0',
        'Closed-canopy deciduous and semi-deciduous forests in warm seasonally wet/dry climates, diverse life forms\xa0',
        'Global wet/dry tropics & subtropics\xa0'],
       ['T1.3', 'T1.3 Tropical-subtropical montane rainforests\xa0',
        'Closed-canopy evergreen forests with abundant non-vascular epiphytes in warm/cool wet cloudy climates, diverse life forms\xa0',
        'Global tropical & subtropical mountains\xa0'],
       ['T1.4', 'T1.4 Tropical heath forests\xa0',
        'Low closed-canopy evergreen forests in warm wet climates on low-nutrient substrates, structurally simple cf T1\xa0',
        'Amazon basin, southeast Asia, possibly Congo basin\xa0'